In [2]:
import os
import pickle
from scipy import signal
import numpy as np
import seaborn as sns
from scipy import stats
import pandas as pd
from tqdm import tqdm
from load_djembe_marker import *
from foot_module import onset_calculations, onset_extraction, onset_filtering, utils, onset_plot
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, argrelmin, argrelmax
from pydub import AudioSegment
from scipy.signal import find_peaks
from pydub.generators import Triangle

from compute_tempo import *
from dance_evaluation import *
from mutils import DataHandler

In [ ]:
# mvnfiles = ["BKO_E1_D1_01_Suku_T.mvnx", "BKO_E1_D1_02_Maraka_T.mvnx", "BKO_E1_D1_03_Wasulunka_T.mvnx", "BKO_E1_D2_04_Maraka_T.mvnx"]

# filename = mvnfiles[2]
# piece_name = os.path.basename(filename).split(".")[0]
# data_handler = DataHandler()
# motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm = data_handler.load_and_process_data(filename, mode = 'gr', drum = 'J2', section_idx=0)

### One Sensor

In [3]:
# SEGMENT_HEAD  SEGMENT_PELVIS   SEGMENT_T8  SEGMENT_LEFT_HAND  SEGMENT_LEFT_FOOT
pkl_filelist = os.listdir(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2")
data_handler = DataHandler()

bpm_recording = []
modes = ["zero_uni", "zero_bi", "p1", "p2"]
for mode in modes:
    result = {
    "piece": [],
    "bpm": [],
    "X_a": [],
    "Y_a": [],
    "Z_a": [],
    # "X_b": [],
    # "Y_b": [],
    # "Z_b": [],
    "bpm_mode": [],
    "bpm_median": [],
    "mode_x": [],
    "mode_y": [],
    "mode_z": [],
    "median_x": [],
    "median_y": [],
    "median_z": [],
    }
    for idx, filename in enumerate(pkl_filelist):

        filename = filename.replace("_Dancers.csv", "_T.mvnx")
        piece_name = os.path.basename(filename).split(".")[0]
        try:
            motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm = data_handler.load_and_process_data(filename, mode = 'gr', drum = 'J2', section_idx=0)
        except ValueError as e:
            # Handle the error and continue
            print(f"Error encountered for: {e}")
            continue

        duration = int(end_t-start_t)
        w_sec = int(duration)
        h_sec = int(w_sec/4)

        mocap_fps = 240
        window_size = int(240*w_sec)
        hop_size = int(240*h_sec)
        tempi_range = np.arange(70,145,1)   # good: 70,145   

        sensorA_velocity = motion_data['velocity']["SEGMENT_LEFT_FOOT"][start_f:end_f, :]
        sensorA_position = motion_data['position']["SEGMENT_LEFT_FOOT"][start_f:end_f, :]

        sensorA_velocity_norm = z_score_normalize(sensorA_velocity)
        sensorA_position_norm = z_score_normalize(sensorA_position)
        
        sensorA_positionde = detrend_signal_array(sensorA_position_norm, cutoff= 0.5)
        sensorA_positionde = smooth_velocity(sensorA_positionde, abs="no") # size (n, 3)
        
        novelty_length = len(sensorA_velocity)
        time_axis = np.arange(novelty_length)/mocap_fps
        bpm_axes = []
        mag_axes = []
        for ax in range(3):
            sensorA_vel = sensorA_velocity_norm[:, ax].reshape(-1,1)
            sensorA_pos = sensorA_positionde[:, ax].reshape(-1,1)
            
            tempo_json_one_sensor = main_one_sensor_peraxis(sensorA_vel, sensorA_pos, 
                                                            mocap_fps, window_size, hop_size, tempi_range, 
                                                            distance_threshold=0.1, T_filter=0.25, vel_thres = 0.05,
                                                            mode=mode)

            # sensor_abs_vel = tempo_json_one_sensor["sensor_abs_vel"]
            # sensor_dir_onsets = tempo_json_one_sensor["sensor_dir_change_onsets"]
            # sensor_dir_onsets_f = tempo_json_one_sensor["sensor_dir_change_onsets_f"]
            sensor_onsets = tempo_json_one_sensor["sensor_onsets"]

            # tempogram_ab = tempo_json_one_sensor["tempogram_ab"]
            # time_axis_seconds = tempo_json_one_sensor["time_axis_seconds"]
            # tempo_axis_bpm = tempo_json_one_sensor["tempo_axis_bpm"]

            tempo_data_maxmethod = tempo_json_one_sensor["tempo_data_maxmethod"]
            tempo_data_weightedkernel = tempo_json_one_sensor["tempo_data_weightedkernel"]
            tempo_data_topN = tempo_json_one_sensor["tempo_data_topN"]

            # Max method
            # Aestimated_beat_pulse = tempo_data_maxmethod["estimated_beat_pulse"]
            # Atempo_curve = tempo_data_maxmethod["tempo_curve"]
            
            bpmA_arr = tempo_data_maxmethod["bpm_arr"]
            magA_arr = tempo_data_maxmethod["mag_arr"]
            tempo_A = np.round(np.average(bpmA_arr), 2)
            # tempo_B = np.round(np.average(bpmB_arr), 2)


            # Weighted method
            # Bestimated_beat_pulse = tempo_data_weightedkernel["estimated_beat_pulse"]
            # Btempo_curve = tempo_data_weightedkernel["tempo_curve"]
            # bpmB_arr = tempo_data_weightedkernel["bpm_arr"]
            # Btempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]
            # Bglobal_tempo_bpm = tempo_data_weightedkernel["global_tempo_bpm"]

            # combined_axis_onsets = np.sum(sensor_dir_onsets, axis=1)     # combine the onsets from 3 axes
            # combined_axis_onsets = np.where(combined_axis_onsets > 0, 1,0)
            # combined_axis_onsets = filter_dir_onsets_by_threshold(combined_axis_onsets.reshape(-1,1), threshold_s= 0.25)
            # combined_axis_onsets = combined_axis_onsets.flatten()   # Binary onsets

            # dance_onset, estimated_beat_onset, drum_ref, dance_onset_iois, estimated_beats_iois = data_handler.onsets_for_plotting(sensor_dir_onsets_f, Aestimated_beat_pulse, novelty_length)
            # dance_bpm = data_handler.calc_tempo_from_onsets(dance_onset)



            bpm_axes.append(bpmA_arr)
            
            
            mode_x = stats.mode(bpmA_arr.flatten())[0]
            mode_y = stats.mode(bpmA_arr.flatten())[0]
            mode_z = stats.mode(bpmA_arr.flatten())[0]

            median_x = np.median(bpmA_arr.flatten())
            median_y = np.median(bpmA_arr.flatten())
            median_z = np.median(bpmA_arr.flatten())
            
            if ax == 0:
                result["piece"].append(piece_name)
                result["bpm"].append(np.round(bpm,2))
                
                result["mode_x"].append(mode_x)
                result["mode_y"].append(mode_y)
                result["mode_z"].append(mode_z)
                
                result["median_x"].append(median_x)
                result["median_y"].append(median_y)
                result["median_z"].append(median_z)
                
                result["X_a"].append(tempo_A)
                # result["X_b"].append(tempo_B)
            elif ax == 1:
                result["Y_a"].append(tempo_A)
                # result["Y_b"].append(tempo_B)
            elif ax == 2:
                result["Z_a"].append(tempo_A)
                # result["Z_b"].append(tempo_B)

            print(f"bpmA_{ax}:", bpmA_arr)
            print("Average A:", tempo_A)                           # [start_f:end_f]

            
            mag_axes.append(magA_arr)
            mag_axes_arr = np.column_stack(mag_axes)    # n by 3 array
            
        bpm_axes_arr = np.column_stack(bpm_axes)    # n by 3 array
        bpm_mode = stats.mode(bpm_axes_arr.flatten())[0]
        bpm_median = np.median(bpm_axes_arr.flatten())
        result["bpm_mode"].append(bpm_mode)
        result["bpm_median"].append(bpm_median)
        
        bpm_recording.append((bpm, bpm_axes_arr, mag_axes_arr))

            
    results_df = pd.DataFrame(result)
    csv_filename = f"./results/vel_1s/foot/peraxis/foot_peraxis_{mode}_70_145.csv"
    results_df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")  

# 94 115 114

Loaded BKO_E1_D1_08_Suku_T.pkl
Error encountered for: Group 'gr' not found in the dataset.
Loaded BKO_E3_D5_03_Wasulunka_T.pkl
Total Sections: 1
bpmA_0: [110. 111.  78. 139.  82.]
Average A: 104.0
bpmA_1: [120. 120. 121.  80.  80.]
Average A: 104.2
bpmA_2: [139. 139.  80.  81. 132.]
Average A: 114.2
Loaded BKO_E2_D4_05_Sandia_T.pkl
Total Sections: 1
bpmA_0: [ 74.  74. 114.  70.  75.]
Average A: 81.4
bpmA_1: [144. 144.  76.  72.  72.]
Average A: 101.6
bpmA_2: [100. 100.  70.  70.  71.]
Average A: 82.2
Loaded BKO_E2_D4_01_Suku_T.pkl
Total Sections: 1
bpmA_0: [98. 97. 93. 93. 94.]
Average A: 95.0
bpmA_1: [129. 130. 129.  80.  80.]
Average A: 109.6
bpmA_2: [129. 129. 129. 129. 129.]
Average A: 129.0
Loaded BKO_E2_D4_03_Wasulunka_T.pkl
Total Sections: 1
bpmA_0: [ 90.  89. 109. 108. 108.]
Average A: 100.8
bpmA_1: [113. 109. 136. 109. 110.]
Average A: 115.4
bpmA_2: [ 75. 142. 109. 109. 110.]
Average A: 109.0
Loaded BKO_E1_D2_04_Maraka_T.pkl
Total Sections: 1
bpmA_0: [101.  88. 112. 112. 110.]

In [ ]:
# bpm_axes_arr is 4by3 array with rows: bpm value for frames and columns: x,y z 

mode_list = []
median_list = []
for i in range(len(bpm_recording)):
    # i = 10
    bpm = bpm_recording[i][0]
    bpm_axes_arr = bpm_recording[i][1]
    mag_axes_arr = bpm_recording[i][2]
    bpm_flat = bpm_axes_arr.flatten()
    sort_idx = np.argsort(bpm_axes_arr.flatten())
    min_n = bpm_flat[sort_idx[:3]]

    # error1 = [np.log2(bpm_axes_arr[j,:] / min_n[0]) for j in range(bpm_axes_arr.shape[0])]
    # error1_ar = np.vstack(np.round(error1,2))

    modes = np.array([
    stats.mode(bpm_axes_arr[:, 0], axis=None)[0],
    stats.mode(bpm_axes_arr[:, 1], axis=None)[0],
    stats.mode(bpm_axes_arr[:, 2], axis=None)[0],
    stats.mode(bpm_axes_arr, axis=None)[0]
    ])

    medians = np.array([
        np.median(bpm_axes_arr[:, 0]),
        np.median(bpm_axes_arr[:, 1]),
        np.median(bpm_axes_arr[:, 2]),
        np.median(bpm_axes_arr)
    ])
    
    mode_list.append(modes)
    median_list.append(medians)
    print("\nRef BPM:",round(bpm))
    # print("Mode BPM X:",stats.mode(bpm_axes_arr[:,0].flatten())[0])
    # print("Mode BPM Y:",stats.mode(bpm_axes_arr[:,1].flatten())[0])
    # print("Mode BPM Z:",stats.mode(bpm_axes_arr[:,2].flatten())[0])
    print("Mode BPM XYZ:",stats.mode(bpm_axes_arr.flatten())[0])

    # print("\nMedian BPM X:", np.median(bpm_axes_arr[:,0].flatten()))
    # print("Median BPM Y:", np.median(bpm_axes_arr[:,1].flatten()))
    # print("Median BPM Z:", np.median(bpm_axes_arr[:,2].flatten()))
    print("Median BPM XYZ:", np.median(bpm_axes_arr.flatten()))
    
    # print("\nMode BPM F1:",stats.mode(bpm_axes_arr[0,:].flatten())[0])
    # print("Mode BPM F2:",stats.mode(bpm_axes_arr[1,:].flatten())[0])
    # print("Mode BPM F3:",stats.mode(bpm_axes_arr[2,:].flatten())[0])
    # print("Mode BPM F4:",stats.mode(bpm_axes_arr[3,:].flatten())[0])

    

In [ ]:
bpm_recording[14][1]

In [ ]:
print(bpm)
print(bpm_axes_arr)
print(error1_ar)

In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(sensor_onsets)


plt.figure(figsize=(40, 6), dpi=200)
plt.plot(Atempo_curve[:len(sensor_onsets)])

In [ ]:
tempogram_ab, tempogram_raw, time_axis_seconds, tempo_axis_bpm = compute_tempogram(sensor_abs_vel[start_f:end_f], mocap_fps, 
                                                                     window_length=window_size, hop_size=hop_size, tempi=tempi_range)

fig, axs = plt.subplots(1, 1, figsize=(6,6), dpi=100)

cax1 = axs.pcolormesh(time_axis_seconds, tempo_axis_bpm, tempogram_ab[0], shading='auto', cmap='magma')
axs.set_title('X-axis')
axs.set_xlabel('Time [s]')
axs.set_ylabel('Tempo [BPM]')
plt.colorbar(cax1, ax=axs, orientation='horizontal', label='Magnitude')
plt.show()

In [ ]:
plot_tempogram_perAxis(tempo_json_one_sensor, islog= 'no', dpi=100)

In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(100*Aestimated_beat_pulse[start_f:end_f], linewidth=1, color = 'b')
plt.plot(Atempo_curve[start_f:end_f], linewidth=1, color = 'r')
plt.xlabel('Time (seconds)')
plt.ylabel('Tempo (BPM)')
plt.title(f'{piece_name} Start:{round(start_t)} End:{round(end_t)}')
plt.grid(True)
plt.show()

In [ ]:
pos_b = np.where(sensorA_position[start_f:end_f]> 0, 1,0)
plt.figure(figsize=(40, 6), dpi=300)

plt.plot(sensor_abs_vel[start_f:end_f], linewidth = 1.0, color='b')
plt.plot(pos_b, linewidth = 1.0, color='b')
# plt.plot(sensor_dir_onsets_f[start_f:end_f], linewidth = 1.0, color='r')

In [ ]:
# Per mode: drum onset and directional change onset plot

plt.figure(figsize=(40, 6), dpi=300)
window_size = 0.1
for onset in beat_ref:
    window_start = onset - (window_size/2)  # Start of the window (25ms before)
    window_end = onset + (window_size/2)   # End of the window (25ms after)
    
    # Plot shaded window
    plt.axvspan(window_start, window_end, color='red', alpha=0.3)
    # Plot reference onset as a vertical line
    plt.axvline(onset, color='red', linestyle='--', linewidth=0.9)

plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='g', linewidth=1.2,)
# plt.vlines(x= estimated_beat_onset, ymin=0.0, ymax=1, color='b', linewidth=1.5,)
plt.xlabel('Time (seconds)')
plt.title(f'{piece_name}')
plt.grid(True)
plt.show()

In [ ]:
results = evaluate_dance_onsets_with_half_beats(beat_ref, dance_onset, tolerance=0.2)
print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")
print("-"*40)

In [ ]:
time = np.arange(novelty_length) / mocap_fps
peaks, properties = signal.find_peaks(Aestimated_beat_pulse)  # , prominence=0.02
beat_peaks_sec = time[peaks]

In [ ]:
click_duration = 50  # milliseconds
click_freq = 1200  # Hz
file_name ="maraka"
# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)

onset_times = beat_peaks_sec  # kept_onsets/240   beat_peaks_sec
dN = novelty_length
total_duration = (dN/240)*1000  #  in milliseconds

audio = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio = audio.overlay(click, position=position)

# Export the audio with clicks to a file
audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Both_Foot_new.wav"), format="wav")
# audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Bothhand_dir.wav"), format="wav")